In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/LeNet5")
#自定義資料取得
import ESC50_puliic_getDataset as epg
#自定義模組
import ESC50_puliic_model as epm

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#測試模型對於電鋸資料集的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    val_loss_array = []
    val_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #驗證準確度
        correct_val= 0
        #驗證資料總筆數
        total_val = 0
        
        for data in val_loader:
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #包裝參數 重組形狀
            val = Variable(inputs.view(input_shape))
            #向前傳播
            outputs = model(val)
            #計算loss
            val_loss = loss_func(outputs, labels)
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_val += len(labels)
            #計算判斷正確的筆數
            correct_val += (predicted == labels).float().sum()
        #計算這次驗證的準確度
        val_accuracy = 100 * correct_val / float(total_val)
        #紀錄這次驗證的準確度
        val_accuracy_array.append(val_accuracy)
        #紀錄這次驗證的loss
        val_loss_array.append(val_loss.data)
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, val_loss_array, val_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [3]:
train_loss_array = []
train_accuracy_array = []
val_loss_array = []
val_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]

#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    train_loss_TL = []
    train_accuracy_TL = []
    val_loss_TL = []
    val_accuracy_TL = []
    test1_accuracy_TL = []
    test3_accuracy_TL = []
    test5_accuracy_TL = []
    #建立模型
    model = epm.LeNet(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    
    #資料抓入
    test_loader1 = epg.getTestDataset(sound[0])
    test_loader3 = epg.getTestDataset(sound[1])
    test_loader5 = epg.getTestDataset(sound[2])
    
    for TL in range(2,-1,-1):
        print("----------------------------------------------------------------------------------------------------------------")
        print("TL: ", TL+1, "fold: ", fold)
        print("----------------------------------------------------------------------------------------------------------------")
        #資料抓入
        train, val = epg.getTrainDataset(fold, sound[TL], num_class)
        #分批方法
        train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
        val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)
        #優化器
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        #調度器
        scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        #訓練模型
        train_loss, train_accuracy, val_loss, val_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[TL], num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5)
    
        #將gpu資料取回
        train_loss = torch.stack(train_loss,dim=-1).cpu()
        train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
        val_loss = torch.stack(val_loss,dim=-1).cpu()
        val_accuracy = torch.stack(val_accuracy,dim=-1).cpu()
    
        for i in range(num_epochs):
            # 寫入資料
            f.write("TL :{} fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(TL+1 ,fold, i+1, num_epochs, train_loss[i], train_accuracy[i], val_loss[i], val_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        train_loss_TL.append(train_loss)
        train_accuracy_TL.append(train_accuracy)
        val_loss_TL.append(val_loss)
        val_accuracy_TL.append(val_accuracy)
        test1_accuracy_TL.append(test1_accuracy)
        test3_accuracy_TL.append(test3_accuracy)
        test5_accuracy_TL.append(test5_accuracy)
        
    train_loss_array.append(train_loss_TL)
    train_accuracy_array.append(train_accuracy_TL)
    val_loss_array.append(val_loss_TL)
    val_accuracy_array.append(val_accuracy_TL)
    test1_accuracy_array.append(test1_accuracy_TL)
    test3_accuracy_array.append(test3_accuracy_TL)
    test5_accuracy_array.append(test5_accuracy_TL)
f.close()

----------------------------------------------------------------------------------------------------------------
TL:  3 fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 2.3085083961486816 Train_acc: 10.000000% Val_Loss: 2.3025293350219727 Val_accuracy: 10.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 2/30 Train_Loss: 2.222228527069092 Train_acc: 10.312500% Val_Loss: 2.116142988204956 Val_accuracy: 17.500000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 3/30 Train_Loss: 2.1964004039764404 Train_acc: 13.750000% Val_Loss: 2.310302257537842 Val_accuracy: 13.750000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 4/30 Train_Loss: 2.147331953048

Train Epoch: 3/30 Train_Loss: 1.7808195352554321 Train_acc: 35.729168% Val_Loss: 1.807927131652832 Val_accuracy: 35.833336%
Test accuracy of the model: 75.200000%
Test accuracy of the model: 77.333333%
Test accuracy of the model: 79.000000%
Train Epoch: 4/30 Train_Loss: 1.606870174407959 Train_acc: 38.125004% Val_Loss: 1.765081524848938 Val_accuracy: 36.666668%
Test accuracy of the model: 70.600000%
Test accuracy of the model: 72.666667%
Test accuracy of the model: 70.000000%
Train Epoch: 5/30 Train_Loss: 1.6026504039764404 Train_acc: 36.041668% Val_Loss: 1.7354506254196167 Val_accuracy: 33.333336%
Test accuracy of the model: 68.200000%
Test accuracy of the model: 69.666667%
Test accuracy of the model: 69.000000%
Train Epoch: 6/30 Train_Loss: 1.5656280517578125 Train_acc: 38.020836% Val_Loss: 1.6836107969284058 Val_accuracy: 34.583336%
Test accuracy of the model: 73.200000%
Test accuracy of the model: 75.666667%
Test accuracy of the model: 76.000000%
Train Epoch: 7/30 Train_Loss: 1.486

Test accuracy of the model: 82.000000%
Train Epoch: 6/30 Train_Loss: 1.3674952983856201 Train_acc: 47.687500% Val_Loss: 1.6115971803665161 Val_accuracy: 40.250000%
Test accuracy of the model: 71.800000%
Test accuracy of the model: 72.333333%
Test accuracy of the model: 71.000000%
Train Epoch: 7/30 Train_Loss: 1.3723477125167847 Train_acc: 47.062500% Val_Loss: 1.4896247386932373 Val_accuracy: 40.500000%
Test accuracy of the model: 72.600000%
Test accuracy of the model: 78.333333%
Test accuracy of the model: 77.000000%
Train Epoch: 8/30 Train_Loss: 1.438886046409607 Train_acc: 48.062500% Val_Loss: 1.489793300628662 Val_accuracy: 39.750000%
Test accuracy of the model: 78.400000%
Test accuracy of the model: 82.666667%
Test accuracy of the model: 86.000000%
Train Epoch: 9/30 Train_Loss: 1.1802955865859985 Train_acc: 48.312500% Val_Loss: 1.8314428329467773 Val_accuracy: 44.250000%
Test accuracy of the model: 69.000000%
Test accuracy of the model: 69.333333%
Test accuracy of the model: 70.000

Test accuracy of the model: 81.333333%
Test accuracy of the model: 84.000000%
Train Epoch: 9/30 Train_Loss: 1.955973744392395 Train_acc: 18.750000% Val_Loss: 1.8718888759613037 Val_accuracy: 28.750000%
Test accuracy of the model: 84.200000%
Test accuracy of the model: 90.000000%
Test accuracy of the model: 90.000000%
Train Epoch: 10/30 Train_Loss: 1.9299206733703613 Train_acc: 28.750000% Val_Loss: 2.05328631401062 Val_accuracy: 26.250000%
Test accuracy of the model: 84.800000%
Test accuracy of the model: 91.000000%
Test accuracy of the model: 93.000000%
Train Epoch: 11/30 Train_Loss: 1.8633917570114136 Train_acc: 28.437500% Val_Loss: 1.936649203300476 Val_accuracy: 31.250000%
Test accuracy of the model: 84.800000%
Test accuracy of the model: 90.333333%
Test accuracy of the model: 92.000000%
Train Epoch: 12/30 Train_Loss: 1.8024582862854004 Train_acc: 30.000000% Val_Loss: 1.7126437425613403 Val_accuracy: 31.250000%
Test accuracy of the model: 84.800000%
Test accuracy of the model: 91.00

Test accuracy of the model: 79.600000%
Test accuracy of the model: 86.333333%
Test accuracy of the model: 90.000000%
Train Epoch: 12/30 Train_Loss: 1.4608454704284668 Train_acc: 45.520836% Val_Loss: 1.3573848009109497 Val_accuracy: 45.416668%
Test accuracy of the model: 77.800000%
Test accuracy of the model: 82.666667%
Test accuracy of the model: 86.000000%
Train Epoch: 13/30 Train_Loss: 1.3281038999557495 Train_acc: 48.437504% Val_Loss: 1.5569344758987427 Val_accuracy: 46.666668%
Test accuracy of the model: 76.600000%
Test accuracy of the model: 81.333333%
Test accuracy of the model: 84.000000%
Train Epoch: 14/30 Train_Loss: 1.2688755989074707 Train_acc: 48.958336% Val_Loss: 1.4217268228530884 Val_accuracy: 45.833336%
Test accuracy of the model: 78.400000%
Test accuracy of the model: 83.333333%
Test accuracy of the model: 87.000000%
Train Epoch: 15/30 Train_Loss: 1.4103376865386963 Train_acc: 49.270836% Val_Loss: 1.3614343404769897 Val_accuracy: 48.333336%
Test accuracy of the model: 

...

Test accuracy of the model: 64.000000%
Test accuracy of the model: 63.666667%
Test accuracy of the model: 60.000000%
Train Epoch: 27/30 Train_Loss: 1.7681879997253418 Train_acc: 35.937500% Val_Loss: 1.9791741371154785 Val_accuracy: 37.500000%
Test accuracy of the model: 65.600000%
Test accuracy of the model: 65.000000%
Test accuracy of the model: 61.000000%
Train Epoch: 28/30 Train_Loss: 1.7193975448608398 Train_acc: 35.625000% Val_Loss: 1.7910360097885132 Val_accuracy: 41.250000%
Test accuracy of the model: 66.400000%
Test accuracy of the model: 65.666667%
Test accuracy of the model: 64.000000%
Train Epoch: 29/30 Train_Loss: 1.7748037576675415 Train_acc: 37.500000% Val_Loss: 1.802211046218872 Val_accuracy: 41.250000%
Test accuracy of the model: 66.400000%
Test accuracy of the model: 67.666667%
Test accuracy of the model: 65.000000%
Train Epoch: 30/30 Train_Loss: 1.8179140090942383 Train_acc: 37.812500% Val_Loss: 2.0566437244415283 Val_accuracy: 40.000000%
Test accuracy of the model: 6

Test accuracy of the model: 77.000000%
Test accuracy of the model: 79.666667%
Test accuracy of the model: 80.000000%
Train Epoch: 30/30 Train_Loss: 1.3671987056732178 Train_acc: 46.250004% Val_Loss: 1.538641095161438 Val_accuracy: 46.250004%
Test accuracy of the model: 79.800000%
Test accuracy of the model: 82.000000%
Test accuracy of the model: 82.000000%
----------------------------------------------------------------------------------------------------------------
TL:  1 fold:  4
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.5764890909194946 Train_acc: 38.312500% Val_Loss: 3.5087292194366455 Val_accuracy: 39.500000%
Test accuracy of the model: 77.600000%
Test accuracy of the model: 81.666667%
Test accuracy of the model: 81.000000%
Train Epoch: 2/30 Train_Loss: 1.398816466331482 Train_acc: 41.312500% Val_Loss: 1.7119468450546265 Val_accuracy: 40.000000%
Test accuracy of the model: 75.60

Test accuracy of the model: 21.200000%
Test accuracy of the model: 16.000000%
Test accuracy of the model: 12.000000%
Train Epoch: 2/30 Train_Loss: 2.135272979736328 Train_acc: 13.750000% Val_Loss: 2.087946653366089 Val_accuracy: 15.000000%
Test accuracy of the model: 63.600000%
Test accuracy of the model: 63.333333%
Test accuracy of the model: 61.000000%
Train Epoch: 3/30 Train_Loss: 2.1644740104675293 Train_acc: 15.937500% Val_Loss: 2.163395881652832 Val_accuracy: 16.250000%
Test accuracy of the model: 64.600000%
Test accuracy of the model: 65.333333%
Test accuracy of the model: 64.000000%
Train Epoch: 4/30 Train_Loss: 2.155477285385132 Train_acc: 20.625000% Val_Loss: 2.156773567199707 Val_accuracy: 25.000000%
Test accuracy of the model: 66.400000%
Test accuracy of the model: 68.333333%
Test accuracy of the model: 67.000000%
Train Epoch: 5/30 Train_Loss: 2.030456304550171 Train_acc: 27.187500% Val_Loss: 2.0944113731384277 Val_accuracy: 26.250000%
Test accuracy of the model: 72.800000%

Test accuracy of the model: 76.800000%
Test accuracy of the model: 79.666667%
Test accuracy of the model: 82.000000%
Train Epoch: 5/30 Train_Loss: 1.6245157718658447 Train_acc: 36.041668% Val_Loss: 1.7738162279129028 Val_accuracy: 33.333336%
Test accuracy of the model: 78.200000%
Test accuracy of the model: 82.000000%
Test accuracy of the model: 87.000000%
Train Epoch: 6/30 Train_Loss: 1.7807587385177612 Train_acc: 35.520836% Val_Loss: 1.6979602575302124 Val_accuracy: 33.333336%
Test accuracy of the model: 75.200000%
Test accuracy of the model: 77.333333%
Test accuracy of the model: 80.000000%
Train Epoch: 7/30 Train_Loss: 1.7389177083969116 Train_acc: 37.916668% Val_Loss: 1.6640725135803223 Val_accuracy: 33.750000%
Test accuracy of the model: 77.200000%
Test accuracy of the model: 80.666667%
Test accuracy of the model: 83.000000%
Train Epoch: 8/30 Train_Loss: 1.5915287733078003 Train_acc: 36.875004% Val_Loss: 1.7173219919204712 Val_accuracy: 31.250002%
Test accuracy of the model: 71.2

Test accuracy of the model: 77.800000%
Test accuracy of the model: 80.333333%
Test accuracy of the model: 82.000000%
Train Epoch: 8/30 Train_Loss: 1.4987616539001465 Train_acc: 44.250000% Val_Loss: 1.285975694656372 Val_accuracy: 43.750000%
Test accuracy of the model: 82.000000%
Test accuracy of the model: 88.333333%
Test accuracy of the model: 89.000000%
Train Epoch: 9/30 Train_Loss: 1.340167760848999 Train_acc: 44.187500% Val_Loss: 1.329206109046936 Val_accuracy: 45.000000%
Test accuracy of the model: 81.200000%
Test accuracy of the model: 86.666667%
Test accuracy of the model: 88.000000%
Train Epoch: 10/30 Train_Loss: 1.465312123298645 Train_acc: 45.750000% Val_Loss: 1.6288036108016968 Val_accuracy: 37.000000%
Test accuracy of the model: 76.400000%
Test accuracy of the model: 79.666667%
Test accuracy of the model: 82.000000%
Train Epoch: 11/30 Train_Loss: 1.2997677326202393 Train_acc: 45.312500% Val_Loss: 2.1701676845550537 Val_accuracy: 44.750000%
Test accuracy of the model: 84.000

In [4]:
#設定存檔位置
f = open("lastData.txt", "w")
for TL in range(3):
    train_loss_last = []
    train_accuracy_last = []
    val_loss_last = []
    val_accuracy_last = []
    test1_accuracy_last = []
    test3_accuracy_last = []
    test5_accuracy_last = []
    
    #整理出訓練完的數值
    for fold in range(5):
        train_loss_last.append(train_loss_array[fold][TL][num_epochs-1])
        train_accuracy_last.append(train_accuracy_array[fold][TL][num_epochs-1])
        val_loss_last.append(val_loss_array[fold][TL][num_epochs-1])
        val_accuracy_last.append(val_accuracy_array[fold][TL][num_epochs-1])
        test1_accuracy_last.append(test1_accuracy_array[fold][TL][num_epochs-1])
        test3_accuracy_last.append(test3_accuracy_array[fold][TL][num_epochs-1])
        test5_accuracy_last.append(test5_accuracy_array[fold][TL][num_epochs-1])
        # 寫入資料
        f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][TL][num_epochs-1], train_accuracy_array[fold][TL][num_epochs-1], val_loss_array[fold][TL][num_epochs-1], val_accuracy_array[fold][TL][num_epochs-1], test1_accuracy_array[fold][TL][num_epochs-1], test3_accuracy_array[fold][TL][num_epochs-1], test5_accuracy_array[fold][TL][num_epochs-1]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    # 寫入資料
    f.write("Max Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(val_loss_last), max(val_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
    f.write("Min Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(val_loss_last), min(val_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
    f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(val_loss_last), np.mean(val_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
f.close()